# 0.Notebook Setup With Hudi Table PII Reader EMR Runtime Role


In [ ]:
%%configure -f
{ "conf": {
    "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar",
    "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.hudi.catalog.HoodieCatalog",
    "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension,com.amazonaws.emr.recordserver.connector.spark.sql.RecordServerSQLExtension",
    "spark.sql.catalog.spark_catalog.lf.managed":"true"
}}

In [ ]:
# Import libraries

import os
from datetime import datetime

from pyspark.sql.functions import col,lit, current_timestamp,unix_timestamp, min, when, desc, split

## 0.1 Global variables setup

Go to 'CloudFormation'. Select the blog stack, and select 'Outputs' tab. Copy 'S3BucketName' value, and replace `<STACK-OUTPUTS-S3-BUCKET-NAME>` in the following cell.

In [ ]:
S3_BUCKET_NAME = <"STACK-OUTPUTS-S3-BUCKET-NAME">

In [ ]:
VERSION = 1

HUDI_CATALOG = "spark_catalog"
TABLE_NAME = "dl_tpc_customer"
HUDI_DATABASE = f"rsv2_blog_hudi_db_{VERSION}"
HUDI_DATABASE_LOCATION = os.path.join(
    "s3://",
    S3_BUCKET_NAME,
    HUDI_DATABASE
)

COW_TABLE_NAME_SQL = f"rsv2_blog_hudi_cow_sql_{TABLE_NAME}_{VERSION}"
COW_TABLE_LOCATION_SQL = os.path.join(
    HUDI_DATABASE_LOCATION,
    COW_TABLE_NAME_SQL
)

MOR_TABLE_NAME_SQL = f"rsv2_blog_hudi_mor_sql_{TABLE_NAME}_{VERSION}"
MOR_TABLE_LOCATION_SQL = os.path.join(
    HUDI_DATABASE_LOCATION,
    MOR_TABLE_NAME_SQL
)

## 0.2 Spark variables setup

In [ ]:
# sparkmagic SQL configs

spark.conf.set('hudi_catalog', HUDI_CATALOG)
spark.conf.set('hudi_db', HUDI_DATABASE)
spark.conf.set('hudi_db_location', HUDI_DATABASE_LOCATION)

spark.conf.set('cow_table_name_sql', COW_TABLE_NAME_SQL)
spark.conf.set('cow_table_location_sql', COW_TABLE_LOCATION_SQL)

spark.conf.set('mor_table_name_sql', MOR_TABLE_NAME_SQL)
spark.conf.set('mor_table_location_sql', MOR_TABLE_LOCATION_SQL)


# 1. Hudi Table PII Reader Lake Formation Configuration

***Please do the following steps in Blog "Query Hudi tables having PII columns" Section before runing the following cells***


# 2. Hudi CoW Table Queries


## 2.1 CoW snapshot query


In [ ]:
%%sql

SELECT 
    c_birth_country, 
    count(*) 
FROM ${hudi_catalog}.${hudi_db}.${cow_table_name_sql} 
GROUP BY c_birth_country;

In [ ]:
%%sql

SELECT
    a.is_masked, count(*)
FROM (
    SELECT 
        CASE 
            WHEN c_first_name = 'MASKED' THEN 'MASKED'
            WHEN c_first_name != 'MASKED' THEN 'NON_MASKED'
        END AS is_masked       
    FROM ${hudi_catalog}.${hudi_db}.${cow_table_name_sql}
    WHERE c_birth_country = 'CHINA'
) a
GROUP BY a.is_masked;

### 2.1.1 CoW snapshot query observations

Comparing with the the same query run in previous **rsv2-hudi-db-creator-notebook:Section 2.2** when first inserting the data to CoW table:
```
HONG KONG   514
CHINA	    486
```
* Partition `HONG KONG` has no change.
* Partition `INDIA` has been added by `INSERT INTO` DML in **Section 2.3**.
* Partition `CHINA` has 3 records been added by `INSERT INTO` DML in **Section 2.3**.


## 2.2 CoW incremental query

Note: Spark SQL using `hudi_table_changes` TVF mentioned on [Hudi website](https://hudi.apache.org/docs/sql_queries#incremental-query) is not supported yet. 

In [ ]:
# Get Hudi commit time

df = spark.sql(f"""
SELECT DISTINCT _hoodie_commit_time FROM {HUDI_CATALOG}.{HUDI_DATABASE}.{COW_TABLE_NAME_SQL} ORDER BY _hoodie_commit_time
""")

commit_ts = [row.asDict()["_hoodie_commit_time"] for row in df.collect()]
commit_ts

We set `begin.instanttime` to the most recent `commit_ts` to see the most recent update of Hudi table. 

In [ ]:
incremental_df = spark.sql(f"""
SELECT * FROM {HUDI_CATALOG}.{HUDI_DATABASE}.{COW_TABLE_NAME_SQL} WHERE _hoodie_commit_time >= {commit_ts[-1]}
""")

incremental_df.createOrReplaceTempView("incremental_view")

In [ ]:
%%sql

SELECT 
    c_birth_country, 
    count(*) 
FROM incremental_view
GROUP BY c_birth_country;


### 2.2.1 CoW incremental query observations

The incremental query shows latest state of Hudi table, where 3 records from Partition `CHINA` and 3 records from Partition `INDIA`. These 6 records are recnetly committed in **rsv2-hudi-db-creator-notebook:Section 2.3**. We cannot see 3 updated records from Partition `HONG KONG` because they have been deleted in **rsv2-hudi-db-creator-notebook:Secton 2.4**

## 2.3 CoW time travel query

Time travel back to the very beginning. 

In [ ]:
# Get Hudi commit time

df = spark.sql(f"""
SELECT DISTINCT _hoodie_commit_time FROM {HUDI_CATALOG}.{HUDI_DATABASE}.{COW_TABLE_NAME_SQL} ORDER BY _hoodie_commit_time
""")

commit_ts = [row.asDict()["_hoodie_commit_time"] for row in df.collect()]
commit_ts

In [ ]:
%%sql

SELECT
    c_birth_country, COUNT(*) as count
FROM ${hudi_catalog}.${hudi_db}.${cow_table_name_sql}
WHERE _hoodie_commit_time IN
(
    SELECT DISTINCT _hoodie_commit_time FROM ${hudi_catalog}.${hudi_db}.${cow_table_name_sql} ORDER BY _hoodie_commit_time LIMIT 1 
)
GROUP BY c_birth_country

### 2.3.1 CoW time travel query observations

The count statistics for each partition match the one in **rsv2-hudi-db-creator-notebook::Section 2.2** when the data has been inserted to Hudi table at the first time.

# 3. Hudi MoR Table Queries

In the context of MoR tables, Apache Hudi creates three logical views of the data, which are represented as three different table names in the Hive metastore

* The first table (e.g. `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}`) is the one with the name we specified when creating the table. This is the *base table* and it stores the original data.
* The second table has the same name as the base table, but with a `_ro` suffix. This is the *Read Optimized (RO)* table. The `_ro` suffix stands for "read-optimized", and this table provides a view of the data that is optimized for read-heavy workloads. It shows the latest compacted data, which means it may not include the most recent updates or changes.
* The third table also has the same name as the base table, but with a `_rt` suffix. This is the *Real-Time (RT)* table. The `_rt` suffix stands for "real-time", and this table provides a view of the data that includes both the compacted data and the most recent updates or changes. This allows for real-time querying of the data

Let's grant the following permissions in Lake Formation

* Grant `Select` and `Describe` permission to Hudi table PII reader role (e.g. ARN: `arn:aws:iam::<ACCOUNT_ID>:role/<STACK_NAME>-hudi-table-pii-role`) for the following tables:
  * `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}` 
  * `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}_ro`
  * `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}_rt`

## 3.1 MoR query on read optimized (RO) table vs. real-time (RT) table

### 3.1.1 After first `INSERT` DML

In **rsv2-hudi-db-creator-notebook:Section 3.3**, we update `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}` table by setting `c_email_address` to `UNKNOWN` for a certain percentage of people born in Hong Kong. 

Let's compare the RO and RT tables. 

In [ ]:
%%sql

SELECT
    a.email_label,
    count(*)
FROM (
    SELECT
        CASE
            WHEN c_email_address = 'UNKNOWN' THEN 'UNKNOWN'
            ELSE 'NOT_UNKNOWN'
        END AS email_label
    FROM ${hudi_catalog}.${hudi_db}.${mor_table_name_sql}_ro
    WHERE c_birth_country = 'HONG KONG'
) a
GROUP BY a.email_label;

In [ ]:
%%sql

SELECT
    a.email_label,
    count(*)
FROM (
    SELECT
        CASE
            WHEN c_email_address = 'UNKNOWN' THEN 'UNKNOWN'
            ELSE 'NOT_UNKNOWN'
        END AS email_label
    FROM ${hudi_catalog}.${hudi_db}.${mor_table_name_sql}_rt
    WHERE c_birth_country = 'HONG KONG'
) a
GROUP BY a.email_label;

#### Observation

The MoR RO table has not captured the `UPDATE` changes like set `c_email_address` to UNKOWN, while RT table has captured the changes where 66 records' `c_email_address` has been set to `UNKNOWN`. 

### 3.1.2 After second `INSERT INTO` DML

In **rsv2-hudi-db-creator-notebook:Section 3.4**, we append 3 records from INDIA and update 3 records' `c_first_name` to `MASK` from CHINA in `rsv2_blog_hudi_db_{VERSION}.rsv2_blog_hudi_mor_sql_dl_tpc_customer_{VERSION}` table. 

Let's compare RO and RT tables

In [ ]:
%%sql

SELECT *  
FROM ${hudi_catalog}.${hudi_db}.${mor_table_name_sql}_ro
WHERE 
    c_birth_country = 'INDIA' OR c_first_name = 'MASKED'

In [ ]:
%%sql

SELECT *  
FROM ${hudi_catalog}.${hudi_db}.${mor_table_name_sql}_rt
WHERE 
    c_birth_country = 'INDIA' OR c_first_name = 'MASKED'

#### Observation

* Both RO and RT tables have append records from INDIA
* RT tables captures the updates from CHINA, but RO tables does not